# Herramientas de apoyo para el análisis y procesado de datos del secuenciador de ADN MinION

**Autor:** Orlandy Ariel Sánchez Acosta.

**Correo:** alu0100773408@ull.edu.es

**Asignatura:** Trabajo de Fin de Grado.

** Este cuaderno se centrará en realizar uno de los filtros utilizados, en este caso el filtrado por calidades.**

In [1]:
FICHERO_ENTRADA = "DatosOriginales/secuencias_originales.fastq" #FICHERO original
#FICHERO_ENTRADA= "DatosOriginales/SRR020192.fastq"
FORMATO = "fastq"

In [2]:
D_PORC_ATIPICOS = 5 #Esta variable será el porcentaje que se quiere quitar de datos atípicos
IT_CONFIANZA = 0.95 # Esta variable será la utilizada para calcular el intervalo de confianza.

In [3]:
FICHERO_SALIDA_MALAS = "Lecturas_malas.fastq"
FICHERO_SALIDA_BUENAS = "Lecturas_buenas.fastq"

In [4]:
PATH_SALIDA_CALIDAD = "Salidas/Calidades/"

## Librerías 
En caso de dudas o cualquier consulta sobre las librerías usadas, consultar los siguientes enlaces.
* [Biopython](http://biopython.org/wiki/Documentation)
* [Plotly](https://plot.ly/python/)
* [Iteratools](https://docs.python.org/2/library/itertools.html)
* [IPython](https://ipython.readthedocs.io/en/stable/install/install.html)
* [Pandas](http://pandas.pydata.org/)
* [NumPy](http://www.numpy.org/)

A continuación, se pasará a importar todas las librerías/módulos que se utilizarán.

In [5]:
### Librería de BioInformática.
from Bio import SeqIO
from Bio.SeqUtils import GC
### Librería para realizar los gráficos 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
### Librerías de utilidades
import numpy as np
from IPython.display import Markdown,display, HTML
from scipy.stats import norm
### Librerías propias
import GraficoDePuntos as gp
import Utilidades as utile
import UtilidadesParaArrays as arrUtile
import PresentacionDatos as psd

In [6]:
IT_CONFIANZA = norm.ppf(IT_CONFIANZA)

In [7]:
c_externa_utile = utile.Utilidades() #llamada a una clase propia para realizar el porcentaje y las permutaciones.
c_arrays_utile = arrUtile.UtilidadesParaArrays()
c_psd = psd.PresentacionDatos()

Definición de variables

In [8]:
dic_cal_buenas = {}#diccionario que guarda, de forma clave:valor, la posición:suma de la calidad de la secuencia buena
dic_cal_malas = {}#diccionario que guarda, de forma clave:valor, la posición:suma de la calidad de la secuencia mala.

## Lecturas de secuencias desde el fichero de entrada
Carga las lecturas del fichero en un array, de manera que solo lea una vez del fichero.

In [9]:
fichero_de_secuencias = [rec for rec in SeqIO.parse(FICHERO_ENTRADA, FORMATO)]

In [10]:
lecturas = [len(rec) for rec in fichero_de_secuencias]

In [11]:
tamanio = len(lecturas)#total de lecturas del fichero
minimo = min(lecturas)#lectrua más pequeña
maximo = max(lecturas)#lectura más grande
c_psd.printmd("* **El fichero: **%s"%FICHERO_ENTRADA)
c_psd.printmd("* **Total de READS (lecturas):** %i"%tamanio)
c_psd.printmd("* **Los READS(lecturas) tiene un tamaño de entre: **%i y %i"%(minimo,maximo))

* **El fichero: **DatosOriginales/secuencias_originales.fastq

* **Total de READS (lecturas):** 1171

* **Los READS(lecturas) tiene un tamaño de entre: **47 y 185942

# Filtro de calidades
Las [calidades](https://en.wikipedia.org/wiki/Phred_quality_score) se indican en la 4ta linea de los ficheros [FASTQ](https://en.wikipedia.org/wiki/FASTQ_format), para obtenerlas BioPython las traduce de manera que nos devuelve un array. ([ir a la documentación de BioPython, apartado "Variables Details"](http://biopython.org/DIST/docs/api/Bio.SeqIO.QualityIO-module.html)).

El filtrado se centrará en analizar las calidades de todas las lecturas y a partir de ellas se sacarán las peores, para su posterior comparación con las ya obtenidas anteriormente.
>Procedimiento para sacar los posibles fallos según las calidades

>* Se obtiene la media y la desviación estándar de las calidades de todas las lecturas.
>* Se aplica _la formula **[ x̄ - 1.64 * σ ]**_ para sacar el límite inferior.
>* Se busca cuales lecturas pasan o no el límite.

In [12]:
sec_a_quitar = c_externa_utile.porcentaje_inverso(D_PORC_ATIPICOS,tamanio)
inicio = sec_a_quitar
fin = (tamanio-inicio)

In [13]:
array_cal_valores = sorted((np.mean(rec.letter_annotations["phred_quality"])) for rec in fichero_de_secuencias)

In [14]:
array_cal_valores = array_cal_valores[inicio:fin]

In [15]:
med_calidad_rec = np.mean(array_cal_valores)
dt_calidad_rec = np.std(array_cal_valores)

In [16]:
limite_inferior_cal = med_calidad_rec - IT_CONFIANZA * dt_calidad_rec

In [17]:
for  i, sec_cal in enumerate(fichero_de_secuencias):   
    med_calidad = np.mean(sec_cal.letter_annotations["phred_quality"])
    if(med_calidad > limite_inferior_cal):
        dic_cal_buenas[i] = med_calidad
    else:
        dic_cal_malas[i] = med_calidad

In [18]:
titulo = "Lectura de secuencias Calidades"
leyenda_b = "% Calidades buenas"
leyenda_m = "% Calidades malas"

In [19]:
porc_bueno_cal = c_externa_utile.tanto_por_ciento(len(dic_cal_buenas),tamanio)

In [20]:
titulo_nombre_fich = FICHERO_ENTRADA.split("/")[1] #extraigo el nombre del fichero

In [21]:
titulo_graf ="<b> %s </b> </br> </br> <b>Fichero:</b> %s </br> %.2f%s </br>%.2f%s"%(titulo,
                                                             titulo_nombre_fich,
                                                             porc_bueno_cal,
                                                             leyenda_b,
                                                             (100-porc_bueno_cal),
                                                             leyenda_m)

In [22]:
grfPtoC = gp.GraficoDePuntos()
graficoC = grfPtoC.crear_grafico_puntos(dic_cal_buenas,
                                        dic_cal_malas,
                                        titulo_graf,
                                        'Posición de la secuencia',
                                        'Media Calidad',
                                        tamanio,
                                        med_calidad_rec,
                                        IT_CONFIANZA,
                                        limite_inferior_cal
                                       )
eliminar_btn_barra  = grfPtoC.eliminar_btn_barra()
plot(graficoC, 
     filename = 'SalidasGraficas/GraficoDePuntosCalidades.html',
     auto_open=False,
     config= eliminar_btn_barra
    )

'file://C:\\Users\\orlan\\Desktop\\TFG\\SalidasGraficas\\GraficoDePuntosCalidades.html'

In [23]:
HTML('<iframe src=SalidasGraficas/GraficoDePuntosCalidades.html width=100% height=550></iframe>')

# Visualizar las secuencias que se detectan como error en las calidades

Para ver las secuencias y comprobar que corresponde con los gráficos, se muestran los datos indicando la posición de la lectura, la secuencia y la media de las calidades.

# Exportar resultados a ficheros FastQ
* **Método de Calidades**
    * Salidas/Calidades/
    
>**NOTA:** Dentro de cada uno de estos directorios se encontrará un fichero FastQ con las lecturas fallidas y otro con las lecturas buenas.

**1.Primero se extraen las posiciones de las lecturas, tanto buenas como fallidas, de los tres métodos.**

In [24]:
a_cal_malas = c_arrays_utile.obtener_claves_dic(dic_cal_malas)
a_cal_buenas = c_arrays_utile.obtener_claves_dic(dic_cal_buenas)

**2.Se extraen las secuencias completas que se encuentran en las posiciones extraídas anteriormente.**

In [25]:
a_sec_cal_malas = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_cal_malas)
a_sec_cal_buenas = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_cal_buenas)

**3.Se procede a guardar las secuencias en los distintos ficheros de salidas.**

In [26]:
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_CALIDAD+FICHERO_SALIDA_MALAS,
                                    FORMATO,
                                    a_sec_cal_malas)
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_CALIDAD+FICHERO_SALIDA_BUENAS,
                                    FORMATO,
                                    a_sec_cal_buenas)